In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})

from datetime import datetime

import numpy as np
import pandas as pd
import math
import os

import tensorflow as tf
np.set_printoptions(threshold=np.inf)

from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import *
from sklearn.metrics import *

import joblib

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def preprocess_data(file_path):
  
    #function that preprocesses the data from the dataset#
    df = pd.read_csv(file_path)
  
    # Remove all NaN-containing entries
    df = df.dropna()
  
    # Interpolate the numerical columns
    numCols = df.select_dtypes(include=np.number).columns.tolist()
    
    df[numCols] = df[numCols].interpolate(method='linear', limit_direction ='forward')
    
    df[numCols] = df[numCols].interpolate(method='linear', limit_direction ='backward') 
    
    
    # Reorder columns
    df = df.reindex(columns=cols)

    # Set index and resample to a 2hour window
    df = df.set_index('Time').resample('1H').median()
    
    # Remove all NaN-containing entries after resampling
    df = df.dropna()
       
    return df

In [ ]:
def transform_data(train, test):
    
    scaler = MinMaxScaler()
    
    train_scaled = scaler.fit_transform(train.iloc[:, :11].values)
    val_scaled = scaler.transform(test.iloc[:, :11].values)
    
    return train_scaled, val_scaled

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In [ ]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

In [ ]:
def create_time_steps(length):
    return list(range(-length, 0))

def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(18, 6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
                 label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def predictWaves(train_data_multi, val_data_multi, x_train_multi, future_steps, EPOCHS, EVALUATION_INTERVAL, PATIENCE):
    multi_step_model = tf.keras.models.Sequential()
    multi_step_model.add(tf.keras.layers.LSTM(64,
                                          return_sequences = True,
                                          input_shape = x_train_multi.shape[-2:]))
    multi_step_model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
    multi_step_model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    multi_step_model.add(tf.keras.layers.LeakyReLU(alpha=0.5)) 
    multi_step_model.add(tf.keras.layers.Dropout(0.2)) 
    multi_step_model.add(tf.keras.layers.LSTM(16, return_sequences=False))
    multi_step_model.add(tf.keras.layers.Dropout(0.2))
    multi_step_model.add(tf.keras.layers.Dense(future_steps))

    print(multi_step_model.summary())
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience = PATIENCE,
                                                  mode='min')
    optimizer = keras.optimizers.Adam(lr=0.001)
    multi_step_model.compile(loss = tf.losses.MeanSquaredError(),
                         optimizer = optimizer,
                         metrics = [tf.metrics.MeanAbsoluteError()])
    
    multi_step_history = multi_step_model.fit(train_data_multi,
                                          epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=EVALUATION_INTERVAL,
                                          callbacks=[early_stopping])
    
    # plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')
    
    histDF = pd.DataFrame(multi_step_history.history)
    
    return multi_step_model, histDF

In [ ]:
def getPredvsAct(test_scaled_01):

    x_test_multi_01, y_test_multi_01 = multivariate_data(test_scaled_01, test_scaled_01[:, 0], 
                                                         0, None, past_lags, future_steps, STEP)

    y_tr = [y[0] for y in y_test_multi_01]
    y_tr_hat = [x[0] for x in waveModel.predict(x_test_multi_01)]

    df_tr_act = pd.concat([pd.DataFrame(y_tr), pd.DataFrame(x_test_multi_01[:,0,1:])],axis=1)
    df_tr_act_inv = scaler.inverse_transform(df_tr_act)
    df_tr_act_invDF = pd.DataFrame(df_tr_act_inv, columns = feature_list[:11])

    df_tr_pred = pd.concat([pd.DataFrame(y_tr_hat), pd.DataFrame(x_test_multi_01[:,0,1:])],axis=1)
    df_tr_pred_inv = scaler.inverse_transform(df_tr_pred)
    df_tr_pred_invDF = pd.DataFrame(df_tr_pred_inv, columns = feature_list[:11])

    times = test_01.reset_index()['Time'].values[-len(y_tr):]
    
    df_tr_pred_act = pd.concat([pd.DataFrame(times), pd.DataFrame(df_tr_pred_invDF['WVHT']), 
                               pd.DataFrame(df_tr_act_invDF['WVHT'])], axis=1)

    df_tr_pred_act.columns = ['Time', 'WVHT_pred', 'WVHT_act']
    
    return df_tr_pred_act, times


In [ ]:
def generate_results_dataset(predictions, ci):
    df = pd.DataFrame()
    df['prediction'] = predictions
    if ci >= 0:
        df['upper'] = predictions + ci
        df['lower'] = predictions - ci
    else:
        df['upper'] = predictions - ci
        df['lower'] = predictions + ci
        
    return df

In [ ]:
def predIntervals(df_tr_pred_act, alpha, times):    
    
    residuals = df_tr_pred_act['WVHT_act'] - df_tr_pred_act['WVHT_pred']

    ci = np.quantile(residuals, 1 - alpha)

    df_conf_01 = generate_results_dataset(pd.DataFrame(df_tr_pred_act['WVHT_pred']), ci)

    df_conf_01 = pd.DataFrame(df_conf_01)

    df_pred_act_01 = pd.concat([pd.DataFrame(times), pd.DataFrame(df_tr_pred_act['WVHT_pred']), 
                               
                                pd.DataFrame(df_tr_pred_act['WVHT_act']), df_conf_01[['upper', 'lower']]], axis=1)

    df_pred_act_01.columns = ['Time', 'WVHT_pred', 'WVHT_act', 'upper', 'lower']

    df_pred_act_index_01 = df_pred_act_01.set_index('Time')
    
    return df_pred_act_index_01, residuals

In [ ]:
def generate_plot(df_pred_act_index_01, residuals):
    
    # calculate prediction metrics
    d = df_pred_act_index_01['WVHT_act'].values - df_pred_act_index_01['WVHT_pred'].values
    mse_f = np.mean(d**2)
    mae_f = np.mean(abs(d))
    rmse_f = np.sqrt(mse_f)
    
    forecastMetrics = {"MAE": mae_f, "MSE": mse_f, "RMSE": rmse_f}

    # Calculate prediction intervals to plot  
    RMSFE = np.sqrt(sum([x**2 for x in residuals]) / len(residuals))

    band_size = 1.96*RMSFE

    fig, ax = plt.subplots(figsize=(15,10))

    ax.plot(df_pred_act_index_01.index, df_pred_act_index_01['WVHT_act'], c='black')

    ax.plot(df_pred_act_index_01.index, df_pred_act_index_01['WVHT_pred'], c='red')

    ax.fill_between(df_pred_act_index_01.index, (df_pred_act_index_01['WVHT_act'] - band_size), 
                    ( df_pred_act_index_01['WVHT_act']+band_size), color='g', alpha=.15)

    ax.set_title(str(future_steps) + "h ahead predictions w/ 95% Confidence")

    ax.set_xlabel('')

    ax.set_ylabel('WVHT (m)')

    ax.legend(('act','pred'), fontsize="15")
    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(15)

    plt.show()
    
    return forecastMetrics

In [ ]:
def predictExt(df, exthresh):
    cv_data = df.copy()

    # Add extreme-value column, with 1 or 0 for > or < exthresh
    cv_data['Extreme_True'] = np.where(cv_data['WVHT'] > exthresh, 1, 0)

    cv_data['Extreme_True'] = cv_data['Extreme_True'].astype('category')

    label = cv_data["Extreme_True"]

    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(label), y = label)

    numCols = cv_data.select_dtypes(include = np.number).columns.tolist()

    for col in numCols: 
        scaler = MinMaxScaler()
        cv_data[col] = scaler.fit_transform(cv_data[col].values.reshape(-1,1))

    X_train, y_train = cv_data.iloc[:, 0 : cv_data.shape[1]-1], label

    ## Define the RF Classifier Model ##
    clf = RandomForestClassifier(n_estimators=1000,   
                                 max_depth=500,
                                 max_features='auto', 
                                 bootstrap=True,
                                 oob_score=True,
                                 n_jobs=-1,
                                 random_state=0,
                                 verbose=0,
                                 warm_start=False,
                                 class_weight='balanced') # class_weights = array([ 0.51540202, 16.73163418])

    ## Fit the model ##
    clf.fit(X_train, y_train)
    
    return clf

In [ ]:
def plotExtCM(cfm_unseen, future_steps, labels, op_path):
    plt.figure(figsize=(12, 8))
    ax = plt.axes()
    sns.set(font_scale = 2)
    sns.heatmap(cfm_unseen, annot=labels, fmt='', cmap='Blues', ax=ax)
    ax.set_title(f"Confusion matrix for {str(future_steps)+'hr ahead'}")
    ax.set_xlabel('Predicted Class',fontweight ='bold', fontsize=20, labelpad=10)
    ax.set_ylabel('True Class',fontweight ='bold', fontsize=20, labelpad=10)
    plt.savefig(op_path + str(future_steps)+'h_RF_CFM.png') 

In [ ]:
### DEFINE PARAMETERS ###

# Reproducibility
SEED = 13
tf.random.set_seed(SEED)

# Data Loader Parameters
BATCH_SIZE = 16
BUFFER_SIZE = 10000
TRAIN_SPLIT = train.shape[0]

# LSTM and PI Parameters
EVALUATION_INTERVAL = 500
EPOCHS = 100
PATIENCE = 5
STEP = 1
past_lags = 120
alpha = 0.05

# user-defined
future_steps = 1
exthresh = 2.0
ip_path = './XWaveNet/Data/'
op_path = './XWaveNet/'

In [ ]:
### MODEL TRAINING ###

## Define feature list ##
feature_list = ['WVHT', 'WSPD', 'WDIR', 'ATMP', 'RH', 'PRES','APD', 'day_cos', 'day_sin', 'month_cos', 'month_sin']

## Read and split data ##
df = preprocess_data(ip_path + 'train_val.csv')

train, test = train_test_split(df, test_size=0.1, shuffle=False)


## Transform data ##
train_scaled, val_scaled = transform_data(train, test)

x_train_multi, train_data_multi, val_data_multi = modelInput(train_scaled, val_scaled, past_lags, future_steps, STEP)

## Forecast wave heights using the LSTM model ##
waveModel, trainHistory = predictWaves(train_data_multi, val_data_multi, x_train_multi, 
                               future_steps, EPOCHS, EVALUATION_INTERVAL, PATIENCE)

## save wave height forecast model ##
waveModel.save(ip_path + str(future_steps) + "h_wavModel.h5")
print('Model Saved!')
 
retrain = 0
if retrain == 0:
    waveModel = load_model(ip_path + str(future_steps) + "h_wavModel.h5")
else:
    waveModel = waveModel        
waveModel.summary()

## Predict Extreme Events ##
extClassifier = predictExt(df, exthresh)

## save extreme event prediction model ##
joblib.dump(extClassifier, op_path + '_extClassifier.joblib')

retrainExt = 0
if retrainExt == 0:
    _extClassifier = joblib.load(op_path + '_extClassifier.joblib') # No need to initialize the model
else:
    _extClassifier = extClassifier

In [ ]:
### PREDICTION ON TEST DATA ###

# Input test data
test_01 = preprocess_data(ip_path + 'test.csv')


# Scale test data
scaler = MinMaxScaler()    
test_scaled_01 = scaler.fit_transform(test_01.iloc[:, :11].values)


# Get Actual vs Predicted values
df_tr_pred_act, times = getPredvsAct(test_scaled_01)


# Get Prediction Intervals
df_pred_act_index_01, residuals = predIntervals(df_tr_pred_act, alpha, times) # get wave height predictions w/ PI


# Prepare test set for extreme event prediction
rfIP = df_pred_act_index_01.copy()
rfIP['Extreme_Forecast'] = np.where(rfIP['WVHT_pred'] > exthresh, 1, 0)
rfIP['Extreme_Forecast'] = rfIP['Extreme_Forecast'].astype('category') # convert forecasted wvht as categorical

rfIP['Extreme_True'] = np.where(rfIP['WVHT_act'] > exthresh, 1, 0)
rfIP['Extreme_True'] = rfIP['Extreme_True'].astype('category') # convert actual wvht as categorical

rfFeatures = test_01.copy()


merged_rfIP = pd.merge(left = rfIP, right = rfFeatures.iloc[:, 1 : rfFeatures.shape[1]], 
                       left_index = True, right_index = True)

merged_rfIP = merged_rfIP.rename(columns={'WVHT_pred':'WVHT'})   

for col in feature_list: 
        scaler = MinMaxScaler()
        merged_rfIP[col] = scaler.fit_transform(merged_rfIP[col].values.reshape(-1,1))
        
        
# Get Extreme Event Predictions from the forecasted values
unseen_predictions = _extClassifier.predict(merged_rfIP[feature_list]) 

# Get Extreme Event Prediction Scores/Metrics
unseen_ras = roc_auc_score(merged_rfIP['Extreme_True'], 
                           _extClassifier.predict_proba(merged_rfIP[feature_list])[:,1])

precision_recall_fscore = precision_recall_fscore_support(merged_rfIP['Extreme_True'].values, 
                                                              unseen_predictions, 
                                                              average='weighted')
accs_unseen = accuracy_score(merged_rfIP['Extreme_True'], unseen_predictions)

cfm_unseen = confusion_matrix(merged_rfIP['Extreme_True'], unseen_predictions)

group_counts = ["{0:0.0f}".format(value) for value in cfm_unseen.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cfm_unseen.flatten()/np.sum(cfm_unseen)]

group_percentages_float = [cfm_unseen.flatten()/np.sum(cfm_unseen)]

group_names = ['True Neg','False Pos','False Neg','True Pos']

labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names, group_counts, group_percentages)]
labels = np.asarray(labels).reshape(2,2)

        
# Save Predictions
preds_op = pd.concat([merged_rfIP['Extreme_True'].reset_index(), 
                      pd.DataFrame(unseen_predictions, columns = [str(future_steps)])], 
                      axis=1)
# preds_op.to_csv(op_path + str(future_steps)+'h_Ext_Predictions.csv')

    
# Compile metrics as a dictionary
exPrediction_metrics = {'ROC_AUC Score': round(unseen_ras, 5), 
             'F1-Score': round(precision_recall_fscore[2], 5), 
             'Accuracy': round(accs_unseen, 5),
             'Precision': round(precision_recall_fscore[0], 5), 
             'Recall': round(precision_recall_fscore[1], 5),
             'FPR': round(group_percentages_float[0][1], 5), 
             'FNR': round(group_percentages_float[0][2], 5)}
    

In [ ]:
# Plot Actual vs Forecasted wave height values from test set

forecastMetrics = generate_plot(df_pred_act_index_01, residuals)

In [ ]:
# Get forecast metrics for test data

forecastMetrics

In [ ]:
# Plot Extreme Event Prediction Confusion Matrix for test data

plotExtCM(cfm_unseen, future_steps, labels, op_path)

In [ ]:
# Get extreme event prediction metrics for test data

exPrediction_metrics